## Preambule

In [1]:
import numpy as np
import pandas as pd
import xarray as xr

In [2]:
path_to_save = "K:/ECEMF/T5.2/Exports_for_figures/"

## Read in general data

In [3]:
xr_2030 = xr.open_dataset("K:/ECEMF/T5.2/PBL/xr_alloc_2030.nc").load()
xr_2040 = xr.open_dataset("K:/ECEMF/T5.2/PBL/xr_alloc_2040.nc").load()
xr_dataread = xr.open_dataset("K:/ECEMF/T5.2/PBL/xr_dataread.nc").load()
xr_nld = xr.open_dataset("K:/ECEMF/T5.2/Allocations/PBL/xr_alloc_NLD.nc").load()
xr_usa = xr.open_dataset("K:/ECEMF/T5.2/Allocations/PBL/xr_alloc_USA.nc").load()
xr_engage = xr.open_dataset("K:/ECEMF/T5.2/xr_policyscen.nc").load()
xr_engage_co2 = xr.open_dataset("K:/ECEMF/T5.2/xr_policyscen_co2.nc").load()

In [4]:
rules = ["GF", "PC", "PCC", "AP", "ECPC", "GDR", "PCB", "PCB_lin"]

## Exports

In [5]:
standard_settings = {}
median_settings = {
    "NonCO2red": 0.5,
    "Convergence_year": 2050,
    "Discount_factor": 0,
    "Historical_startyear": 1990,
    "Capability_threshold": "Th",
    "RCI_weight": "Half",
    "Scenario": "SSP2",
}

#### Schema 1 (general infographic)

In [6]:
emis_1990 = float(xr_dataread.GHG_hist.sel(Region="NLD", Time=1990))
dict_schema_1_h = {}
list_var = list(np.array(xr_dataread.GHG_hist.sel(Region="NLD", Time=np.arange(1990, 2021.5))))
list_time = list(np.arange(1990, 2021.5))
dict_schema_1_h["Time"] = list_time
dict_schema_1_h["GHG_historical"] = list_var
dict_schema_1_h["Red_wrt_1990"] = (emis_1990 - np.array(list_var)) / emis_1990

df_schema_1_h = pd.DataFrame(dict_schema_1_h)
df_schema_1_h.to_excel("K:/ECEMF/T5.2/Exports_for_figures/data_sch1_hist.xlsx", index=False)

In [7]:
emis_2021 = float(xr_dataread.GHG_hist.sel(Region="NLD", Time=2021))
emis_2030_high = 123
emis_2030_low = 97
time = np.arange(2021, 2031)
interp_high = np.interp(time, [2021, 2030], [emis_2021, emis_2030_high])
interp_low = np.interp(time, [2021, 2030], [emis_2021, emis_2030_low])

In [8]:
dict_schema_1_kev = {}
list_time = list(np.arange(2021, 2031))
dict_schema_1_kev["Time"] = list_time
dict_schema_1_kev["GHG_kev_high"] = interp_high
dict_schema_1_kev["GHG_kev_low"] = interp_low

dict_schema_1_kev["GHG_kev_high_red_wrt_1990"] = (emis_1990 - interp_high) / emis_1990
dict_schema_1_kev["GHG_kev_low_red_wrt_1990"] = (emis_1990 - interp_low) / emis_1990

df_schema_1_kev = pd.DataFrame(dict_schema_1_kev)
df_schema_1_kev.to_excel("K:/ECEMF/T5.2/Exports_for_figures/data_sch1_kev.xlsx", index=False)

In [9]:
dict_schema_1_effortsharing = {}
dict_schema_1_effortsharing["Fairness_15_GHG"] = np.array(
    xr_nld.sel(
        Temperature=1.56, Time=2040, Timing="Immediate", NegEmis=0.3, Risk=0.5, **median_settings
    )[rules].to_array()
)
dict_schema_1_effortsharing["Fairness_20_GHG"] = np.array(
    xr_nld.sel(
        Time=2040, Temperature=2.0, Timing="Immediate", Risk=0.33, NegEmis=0.6, **median_settings
    )[rules].to_array()
)
dict_schema_1_effortsharing["Fairness_15_red_1990"] = (
    emis_1990
    - np.array(
        xr_nld.sel(
            Time=2040,
            Temperature=1.56,
            Timing="Immediate",
            NegEmis=0.3,
            Risk=0.33,
            **median_settings,
        )[rules].to_array()
    )
) / emis_1990
dict_schema_1_effortsharing["Fairness_20_red_1990"] = (
    emis_1990
    - np.array(
        xr_nld.sel(
            Time=2040, Temperature=2, Timing="Immediate", Risk=0.33, NegEmis=0.6, **median_settings
        )[rules].to_array()
    )
) / emis_1990
dict_schema_1_effortsharing["Allocation"] = rules

df_schema_1_effortsharing = pd.DataFrame(dict_schema_1_effortsharing)
df_schema_1_effortsharing.to_excel(
    "K:/ECEMF/T5.2/Exports_for_figures/data_sch1_es.xlsx", index=False
)

#### Schema 2 (outline of report)

In [10]:
emis_1990 = float(xr_dataread.GHG_hist.sel(Region="EARTH", Time=1990))
dict_schema_2_h = {}
list_var = list(np.array(xr_dataread.GHG_hist.sel(Region="EARTH", Time=np.arange(1990, 2021.5))))
list_time = list(np.arange(1990, 2021.5))

dict_schema_2_h["Time"] = list_time
dict_schema_2_h["GHG_historical_globe"] = list_var

df_schema_2_h = pd.DataFrame(dict_schema_2_h)
df_schema_2_h.to_excel("K:/ECEMF/T5.2/Exports_for_figures/data_sch2_hist.xlsx", index=False)

In [11]:
emis_1990 = float(xr_dataread.GHG_hist.sel(Region="EARTH", Time=1990))
dict_schema_2_h = {}
list_time = list(np.arange(2021, 2101))
list_var_15 = list(
    np.array(
        xr_dataread.GHG_globe.sel(
            NegEmis=0.4,
            NonCO2red=0.5,
            Temperature=1.5,
            Risk=0.5,
            Timing="Immediate",
            Time=np.arange(2021, 2101),
        )
    )
)
list_var_20 = list(
    np.array(
        xr_dataread.GHG_globe.sel(
            NegEmis=0.4,
            NonCO2red=0.5,
            Temperature=2.0,
            Risk=0.5,
            Timing="Immediate",
            Time=np.arange(2021, 2101),
        )
    )
)
list_var_23 = list(
    np.array(
        xr_dataread.GHG_globe.sel(
            NegEmis=0.4,
            NonCO2red=0.5,
            Temperature=2.3,
            Risk=0.5,
            Timing="Immediate",
            Time=np.arange(2021, 2101),
        )
    )
)

dict_schema_2_h["Time"] = list_time
dict_schema_2_h["GHG_projection_15"] = list_var_15
dict_schema_2_h["GHG_projection_20"] = list_var_20
dict_schema_2_h["GHG_projection_23"] = list_var_23

df_schema_2_h = pd.DataFrame(dict_schema_2_h)
df_schema_2_h.to_excel("K:/ECEMF/T5.2/Exports_for_figures/data_sch2_proj.xlsx", index=False)

#### Figure 2.2 (global pathways)

In [12]:
emis_1990 = float(xr_dataread.GHG_hist.sel(Region="EARTH", Time=1990))
dict_schema_2_h = {}
list_time = list(np.arange(2021, 2101))
list_ghg_15 = list(
    np.array(
        xr_dataread.GHG_globe.sel(
            NegEmis=0.3,
            NonCO2red=0.5,
            Temperature=1.56,
            Risk=0.5,
            Timing="Immediate",
            Time=np.arange(2021, 2101),
        )
    )
)
list_ghg_20 = list(
    np.array(
        xr_dataread.GHG_globe.sel(
            NegEmis=0.6,
            NonCO2red=0.5,
            Temperature=2.0,
            Risk=0.33,
            Timing="Immediate",
            Time=np.arange(2021, 2101),
        )
    )
)
list_co2_15 = list(
    np.array(
        xr_dataread.CO2_globe.sel(
            NegEmis=0.3,
            NonCO2red=0.5,
            Temperature=1.56,
            Risk=0.5,
            Timing="Immediate",
            Time=np.arange(2021, 2101),
        )
    )
)
list_co2_20 = list(
    np.array(
        xr_dataread.CO2_globe.sel(
            NegEmis=0.6,
            NonCO2red=0.5,
            Temperature=2.0,
            Risk=0.33,
            Timing="Immediate",
            Time=np.arange(2021, 2101),
        )
    )
)

dict_schema_2_h["Time"] = list_time

dict_schema_2_h["GHG_15_low"] = list(
    np.array(
        xr_dataread.GHG_globe.sel(
            Temperature=1.56, Risk=0.5, NegEmis=[0.2, 0.3, 0.4], Time=np.arange(2021, 2101)
        ).min(dim=["NonCO2red", "Timing", "NegEmis"])
    )
)
dict_schema_2_h["GHG_15_med"] = list_ghg_15
dict_schema_2_h["GHG_15_high"] = list(
    np.array(
        xr_dataread.GHG_globe.sel(
            Temperature=1.56, Risk=0.5, NegEmis=[0.2, 0.3, 0.4], Time=np.arange(2021, 2101)
        ).max(dim=["NonCO2red", "Timing", "NegEmis"])
    )
)
dict_schema_2_h["CO2_15_low"] = list(
    np.array(
        xr_dataread.CO2_globe.sel(
            Temperature=1.56, Risk=0.5, NegEmis=[0.2, 0.3, 0.4], Time=np.arange(2021, 2101)
        ).min(dim=["NonCO2red", "Timing", "NegEmis"])
    )
)
dict_schema_2_h["CO2_15_med"] = list_co2_15
dict_schema_2_h["CO2_15_high"] = list(
    np.array(
        xr_dataread.CO2_globe.sel(
            Temperature=1.56, Risk=0.5, NegEmis=[0.2, 0.3, 0.4], Time=np.arange(2021, 2101)
        ).max(dim=["NonCO2red", "Timing", "NegEmis"])
    )
)

dict_schema_2_h["GHG_20_low"] = list(
    np.array(
        xr_dataread.GHG_globe.sel(
            Temperature=2.0, Risk=0.33, NegEmis=[0.5, 0.6, 0.7], Time=np.arange(2021, 2101)
        ).min(dim=["NonCO2red", "Timing", "NegEmis"])
    )
)
dict_schema_2_h["GHG_20_med"] = list_ghg_20
dict_schema_2_h["GHG_20_high"] = list(
    np.array(
        xr_dataread.GHG_globe.sel(
            Temperature=2, Risk=0.33, NegEmis=[0.5, 0.6, 0.7], Time=np.arange(2021, 2101)
        ).max(dim=["NonCO2red", "Timing", "NegEmis"])
    )
)
dict_schema_2_h["CO2_20_low"] = list(
    np.array(
        xr_dataread.CO2_globe.sel(
            Temperature=2, Risk=0.33, NegEmis=[0.5, 0.6, 0.7], Time=np.arange(2021, 2101)
        ).min(dim=["NonCO2red", "Timing", "NegEmis"])
    )
)
dict_schema_2_h["CO2_20_med"] = list_co2_20
dict_schema_2_h["CO2_20_high"] = list(
    np.array(
        xr_dataread.CO2_globe.sel(
            Temperature=2, Risk=0.33, NegEmis=[0.5, 0.6, 0.7], Time=np.arange(2021, 2101)
        ).max(dim=["NonCO2red", "Timing", "NegEmis"])
    )
)

df_schema_2_h = pd.DataFrame(dict_schema_2_h)
df_schema_2_h.to_excel("K:/ECEMF/T5.2/Exports_for_figures/data_fig_2_2.xlsx", index=False)

In [13]:
xr_use = xr_engage.CurPol.sel(Region="EARTH")
xr_use_co2 = xr_engage_co2.CurPol.sel(Region="EARTH")

dict_eng = {}
dict_eng["Time"] = np.array(xr_use.Time)
dict_eng["Mean_GHG"] = np.array(xr_use.mean(dim="Model"))
dict_eng["Max_GHG"] = np.array(xr_use.max(dim="Model"))
dict_eng["Min_GHG"] = np.array(xr_use.min(dim="Model"))
dict_eng["Mean_CO2"] = np.array(xr_use_co2.mean(dim="Model"))
dict_eng["Max_CO2"] = np.array(xr_use_co2.max(dim="Model"))
dict_eng["Min_CO2"] = np.array(xr_use_co2.min(dim="Model"))
df_global_pathways = pd.DataFrame(dict_eng)
df_global_pathways.to_excel(
    "K:/ECEMF/T5.2/Exports_for_figures/data_fig_2_2_curpol.xlsx", index=False
)

In [14]:
xr_use = xr_dataread.GHG_hist.sel(Region="EARTH")
xr_use_co2 = xr_dataread.CO2_hist.sel(Region="EARTH")

dict_eng = {}
dict_eng["Time"] = np.array(xr_use.Time)
dict_eng["GHG_hist"] = np.array(xr_use)
dict_eng["CO2_hist"] = np.array(xr_use_co2)
df_global_pathways = pd.DataFrame(dict_eng)
df_global_pathways.to_excel("K:/ECEMF/T5.2/Exports_for_figures/data_fig_2_2_hist.xlsx", index=False)

#### Figure 4.1 (Netherlands zoom)

In [15]:
dict_fig_4_1 = {}
list_rule = []
list_med = []
list_min = []
list_max = []
list_med2 = []
list_min2 = []
list_max2 = []
list_time = []
for rule_i, rule in enumerate(rules):
    val_med = np.array(
        xr_nld.sel(
            Temperature=1.56,
            Risk=0.5,
            NegEmis=0.3,
            Timing="Immediate",
            **median_settings,
            **standard_settings,
        )[rule]
    )
    val_min = np.array(
        xr_nld.sel(Temperature=1.56, Risk=0.5, NegEmis=0.3, **standard_settings)
        .min(dim=median_settings.keys())
        .min(dim="Timing")[rule]
    )
    val_max = np.array(
        xr_nld.sel(Temperature=1.56, Risk=0.5, NegEmis=0.3, **standard_settings)
        .max(dim=median_settings.keys())
        .max(dim="Timing")[rule]
    )
    val_med2 = np.array(
        xr_nld.sel(
            Temperature=2,
            Risk=0.33,
            NegEmis=0.6,
            Timing="Immediate",
            **median_settings,
            **standard_settings,
        )[rule]
    )
    val_min2 = np.array(
        xr_nld.sel(Temperature=2, Risk=0.33, NegEmis=0.6, **standard_settings)
        .min(dim=median_settings.keys())
        .min(dim="Timing")[rule]
    )
    val_max2 = np.array(
        xr_nld.sel(Temperature=2, Risk=0.33, NegEmis=0.6, **standard_settings)
        .max(dim=median_settings.keys())
        .max(dim="Timing")[rule]
    )
    list_rule += [rule] * len(val_med)
    list_med += list(val_med)
    list_max += list(val_max)
    list_min += list(val_min)
    list_med2 += list(val_med2)
    list_max2 += list(val_max2)
    list_min2 += list(val_min2)
    list_time += list(np.array(xr_nld.Time))
dict_fig_4_1["Rule"] = list_rule
dict_fig_4_1["Median_15"] = list_med
dict_fig_4_1["Minimum_15"] = list_min
dict_fig_4_1["Maximum_15"] = list_max
dict_fig_4_1["Median_20"] = list_med2
dict_fig_4_1["Minimum_20"] = list_min2
dict_fig_4_1["Maximum_20"] = list_max2
dict_fig_4_1["Time"] = list_time

df_fig_4_1 = pd.DataFrame(dict_fig_4_1)
df_fig_4_1.to_excel("K:/ECEMF/T5.2/Exports_for_figures/data_fig_4_1.xlsx", index=False)

#### Figure 4.2 (temperature levels on horizontal)

(The diagonal is used to obtain specific values of NegEmis per temperature level)

In [16]:
emis_1990 = float(xr_dataread.GHG_hist.sel(Region="NLD", Time=1990))

dict_fig_4_2 = {}
list_rule = []
list_med = []
list_min = []
list_max = []
list_temp = []
for rule_i, rule in enumerate(rules):
    val_med = (
        -(
            np.diag(
                xr_nld.sel(
                    Time=2040,
                    Risk=0.5,
                    Timing="Immediate",
                    NegEmis=[0.3, 0.3, 0.4, 0.4, 0.5, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6],
                    **median_settings,
                    **standard_settings,
                )[rule]
            )
            - emis_1990
        )
        / emis_1990
    )  # This of course means that the 2.0 deg does not fully fit the other figures!
    val_min = (
        -(
            np.diag(
                xr_nld.sel(Time=2040, Risk=0.5, NonCO2red=[0.33, 0.5, 0.67], **standard_settings)
                .min(dim=median_settings.keys())[rule]
                .min(dim="Timing")
                .sel(NegEmis=[0.3, 0.3, 0.4, 0.4, 0.5, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6])
            )
            - emis_1990
        )
        / emis_1990
    )
    val_max = (
        -(
            np.diag(
                xr_nld.sel(Time=2040, Risk=0.5, NonCO2red=[0.33, 0.5, 0.67], **standard_settings)
                .max(dim=median_settings.keys())[rule]
                .max(dim="Timing")
                .sel(NegEmis=[0.3, 0.3, 0.4, 0.4, 0.5, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6])
            )
            - emis_1990
        )
        / emis_1990
    )
    list_rule += [rule] * len(val_med)
    list_med += list(val_med)
    list_max += list(val_max)
    list_min += list(val_min)
    list_temp += list(np.array(xr_nld.Temperature))
dict_fig_4_2["Rule"] = list_rule
dict_fig_4_2["Median"] = list_med
dict_fig_4_2["Minimum"] = list_min
dict_fig_4_2["Maximum"] = list_max
dict_fig_4_2["Temperature"] = list_temp

df_fig_4_2 = pd.DataFrame(dict_fig_4_2)
df_fig_4_2.to_excel("K:/ECEMF/T5.2/Exports_for_figures/data_fig_4_2.xlsx", index=False)

#### Figure 4.3 (global scale)

In [17]:
emis_2021_all = xr_dataread.GHG_hist.sel(Time=2021)
emis_1990_all = xr_dataread.GHG_hist.sel(Time=1990)

red_2040_2021 = (-(xr_2040 - emis_2021_all) / emis_2021_all).sel(
    Temperature=1.56, Risk=0.5, NegEmis=0.3, **standard_settings
)
red_2021_1990 = -(emis_2021_all - emis_1990_all) / emis_1990_all
median_settings = {
    "NonCO2red": 0.5,
    "Convergence_year": 2050,
    "Discount_factor": 0,
    "Historical_startyear": 1990,
    "Capability_threshold": "Th",
    "RCI_weight": "Half",
    "Scenario": "SSP2",
    "Timing": "Immediate",
}

In [18]:
dict_fig_4_3 = {}
list_rule = []
list_med = []
list_min = []
list_max = []
list_region = []
for rule_i, rule in enumerate(rules):
    val_med = np.array(red_2040_2021.sel(**median_settings)[rule])
    val_min = np.array(red_2040_2021.min(dim=median_settings.keys())[rule])
    val_max = np.array(red_2040_2021.max(dim=median_settings.keys())[rule])
    list_rule += [rule] * len(val_med)
    list_med += list(val_med)
    list_max += list(val_max)
    list_min += list(val_min)
    list_region += list(np.array(red_2040_2021.Region))
dict_fig_4_3["Rule"] = list_rule
dict_fig_4_3["Median"] = list_med
dict_fig_4_3["Minimum"] = list_min
dict_fig_4_3["Maximum"] = list_max
dict_fig_4_3["Region"] = list_region

df_fig_4_3 = pd.DataFrame(dict_fig_4_3)
df_fig_4_3.to_excel("K:/ECEMF/T5.2/Exports_for_figures/data_fig_4_3_2040red.xlsx", index=False)

In [19]:
dict_fig_4_3 = {}
list_rule = []
list_med = []
list_region = []
val_med = np.array(red_2021_1990)
list_med += list(val_med)
list_region += list(np.array(red_2021_1990.Region))
dict_fig_4_3["GHG_wrt_1990"] = list_med
dict_fig_4_3["Region"] = list_region

df_fig_4_3 = pd.DataFrame(dict_fig_4_3)
df_fig_4_3.to_excel("K:/ECEMF/T5.2/Exports_for_figures/data_fig_4_3_2021red.xlsx", index=False)

In [29]:
emis_2021_pc = (1e6 * emis_2021_all) / (
    1e6 * xr_dataread.Population.sel(Time=2021, Scenario="SSP2")
)

dict_fig_4_3 = {}
list_rule = []
list_med = []
list_region = []
val_med = np.array(emis_2021_pc)
list_med += list(val_med)
list_region += list(np.array(emis_2021_pc.Region))
dict_fig_4_3["GHGpc"] = list_med
dict_fig_4_3["Region"] = list_region

df_fig_4_3 = pd.DataFrame(dict_fig_4_3)
df_fig_4_3.to_excel("K:/ECEMF/T5.2/Exports_for_figures/data_fig_4_3_2021pc.xlsx", index=False)